In [57]:
import pandas as pd 
# default='warn', Used here to supress warning when slicing and creating high earners column
pd.options.mode.chained_assignment = None 


In [58]:
import requests
import json
import pandas as pd 
import numpy as np
import math

In [59]:
# custom function from Web_scrapper_League_Tables notebook 
from ipynb.fs.full.Web_scrapper_League_Tables import League_table_scraper

#### Get data from API

In [60]:
API_key='d15f7132973d404d9d046934398ee9e3'

In [61]:
# API authorization
headers = {'x-api-key':'d15f7132973d404d9d046934398ee9e3', 'accept': 'application/json', 'accept': 'text/csv'}

# Choose endpoint
endpoint = 'https://www.capology.com/api/v2/soccer/salaries/br/brasileiro/2020/'

# Retrieve json
json = requests.get(endpoint, headers=headers).text

#Data for 2021
df1 = pd.read_json(json)

df1 = df1[df1["season_term"] == "combined"].reset_index(drop = True)

In [62]:
# API authorization
headers = {'x-api-key':'d15f7132973d404d9d046934398ee9e3', 'accept': 'application/json', 'accept': 'text/csv'}

# Choose endpoint
endpoint = 'https://www.capology.com/api/v2/soccer/salaries/br/brasileiro/2021/'

# Retrieve json
json2 = requests.get(endpoint, headers=headers).text

#Data for 2021
df2 = pd.read_json(json2)

df2 = df2[df2["season_term"] == "summer"]

In [63]:
# API authorization
headers = {'x-api-key':'d15f7132973d404d9d046934398ee9e3', 'accept': 'application/json', 'accept': 'text/csv'}

# Choose endpoint
endpoint = 'https://www.capology.com/api/v2/soccer/salaries/br/brasileiro/2022/'

# Retrieve json
json3 = requests.get(endpoint, headers=headers).text

#Data for 2022
df3 = pd.read_json(json3)

df3 = df3[df3["season_term"] == "summer"]

In [64]:
df = pd.concat([df1,df3,df2],axis=0).drop_duplicates()

#### Clean dataframe

In [65]:
df= df[df['salary_gross_eur'] != 0]

In [66]:
df=df.drop_duplicates().reset_index(drop = True)

In [67]:
CODE_DICT=pd.Series(df.club_code.values,index=df.club_name).to_dict()


In [68]:
#Required columns
LIST=['season_id','club_code','player_age','player_country_id','player_group_code','salary_gross_eur']

In [69]:
df_salary=df[LIST]

#### Add needed features


In [70]:
# Group the data by 'club' and 'position'
groups = df_salary.groupby(['club_code', 'player_group_code','season_id'])
# Find the highest two values of column 'salary_gross_eur' for each group
top_two = groups['salary_gross_eur'].nlargest(2).reset_index()

# get needed indexes
index=list(top_two['level_3'])

# Add the result to the data frame as a new column
df_salary['high_earner'] = 0
for i in index:
    df_salary.loc[i, 'high_earner'] = 1

In [71]:
# Add the result to the data frame as a new column
df['high_earner'] = 0
for i in index:
    df.loc[i, 'high_earner'] = 1

In [72]:
df_salary.head()

,season_id,club_code,player_age,player_country_id,player_group_code,salary_gross_eur,high_earner
0,2020,CAP,20,brazil,D,57155,0
1,2020,CAP,22,brazil,K,76207,0
2,2020,CAP,21,brazil,K,28578,0
3,2020,CAP,20,brazil,F,57155,0
4,2020,CAP,23,brazil,K,85733,0


In [73]:
# Required columns for related players
LIST2=['season_id','player_name','club_code','player_age','player_country_id','player_group_code','salary_gross_eur','high_earner']

In [74]:
df_similar=df[LIST2]

In [75]:
def set_region(row):
    """
    Function takes each row as argument and returns the region 
    based on the player_country_id.
    """
    
    south_america = {'uruguay', 'colombia', 'argentina', 'ecuador','chile', 'venezuela', 'paraguay','peru','bolivia'}
    europe = {'spain','italy','belarus','portugal','finland','ukraine','finland'}
    rest_of_world = {'south-korea','united-states','china','japan','cote-d-ivoire'}
    
    if row['player_country_id'] in south_america:
        return 'south america'
    
    elif row['player_country_id'] in europe:
        return 'europe'
    
    elif row['player_country_id'] in rest_of_world:
        return 'rest of the world'
    
    else:
        return 'brazil'

In [76]:
CODE_DICT

{'Athletico Paranaense': 'CAP',
 'Atletico Mineiro': 'CAM',
 'Botafogo': 'BOT',
 'Ceara Sporting Club': 'CEA',
 'Corinthians': 'CRS',
 'Coritiba': 'CRB',
 'Esporte Clube Bahia': 'ECB',
 'Flamengo': 'FLA',
 'Fluminense': 'FLU',
 'Fortaleza': 'FOR',
 'Goianiense': 'GOE',
 'Goias': 'GOI',
 'Gremio': 'GRM',
 'Internacional': 'INR',
 'Palmeiras': 'PLM',
 'Red Bull Bragantino': 'RED',
 'Santos': 'SAN',
 'Sao Paulo': 'SAP',
 'Sport Club do Recife': 'SCR',
 'Vasco da Gama': 'VAS',
 'America FC': 'AMG',
 'Avai': 'AVI',
 'Cuiaba': 'CUI',
 'Juventude': 'JUD',
 'Chapecoense': 'CPE'}

In [77]:
# Update dictonary by running the map function and checking the mising keys 
CODE_DICT['Grêmio'] = 'GRM'
CODE_DICT['São Paulo'] = 'SAP'
CODE_DICT['Goiás'] = 'GOI'
CODE_DICT['Bahia'] = 'ECB'
CODE_DICT['Ceará'] ='CEA'
CODE_DICT['Atlético Mineiro'] ='CAM'
CODE_DICT['Atlético Goianiense'] ='GOE'
CODE_DICT['Sport'] = 'SCR'
CODE_DICT['América Mineiro'] = 'AMG'
CODE_DICT['Cuiabá'] = 'CUI'
CODE_DICT['Avaí'] = 'AVI'

In [78]:
# Final league standings 2020
standings_2020 = League_table_scraper('https://en.wikipedia.org/wiki/2020_Campeonato_Brasileiro_S%C3%A9rie_A',9)
standings_2020 = list(map(lambda x: CODE_DICT[x], standings_2020.club))

# Final league standings 2021
standings_2021 = League_table_scraper('https://en.wikipedia.org/wiki/2021_Campeonato_Brasileiro_S%C3%A9rie_A',9)
standings_2021 = list(map(lambda x: CODE_DICT[x], standings_2021.club))

# Final league standings 2022
standings_2022 = League_table_scraper('https://en.wikipedia.org/wiki/2022_Campeonato_Brasileiro_S%C3%A9rie_A',9)
standings_2022 = list(map(lambda x: CODE_DICT[x], standings_2022.club))

In [79]:
def set_club_type(row):
    
    """
    Function takes each row as argument and returns
    top 6,6-10,mid-table, relagation based on the club 
    finishing position for the year
    
    """
    if (row['club_code'] in standings_2020[:6] and row['season_id']==2020) or (row['club_code'] in standings_2021[0:6] and row['season_id']==2021) or (row['club_code'] in standings_2022[0:6] and row['season_id']==2022):
        return 'top 6'
   
    elif (row['club_code'] in standings_2020[6:11] and row['season_id']==2020) or (row['club_code'] in standings_2021[6:11] and row['season_id']==2021) or (row['club_code'] in standings_2022[6:11] and row['season_id']==2022):
        return '6-10'
    
    elif (row['club_code'] in standings_2020[11:16] and row['season_id']==2020) or (row['club_code'] in standings_2021[11:16] and row['season_id']==2021) or (row['club_code'] in standings_2022[11:16] and row['season_id']==2022):
        return 'mid-table'
    
    else :
        return 'relegation'


In [80]:
X = df_salary[['club_code', 'player_age', 'player_country_id', 'player_group_code','season_id','high_earner']]

X = X.assign(Age=pd.cut(X['player_age'], 
                               bins=[0, 17, 21,23, 27, 32,99], 
                               labels=['U18', '18-21', '21-23','23-27','27-32','32+']))

X = X.assign(region=X.apply(set_region, axis=1)).astype('category')

X = X.assign(club_level=X.apply(set_club_type,axis=1)).astype('category')

X = X.drop('player_age',axis=1)
 
X = X.drop('season_id',axis=1)

In [81]:
X.head()

,club_code,player_country_id,player_group_code,high_earner,Age,region,club_level
0,CAP,brazil,D,0,18-21,brazil,6-10
1,CAP,brazil,K,0,21-23,brazil,6-10
2,CAP,brazil,K,0,18-21,brazil,6-10
3,CAP,brazil,F,0,18-21,brazil,6-10
4,CAP,brazil,K,0,21-23,brazil,6-10


In [82]:
def Club_type(club):
    
    """
    Function takes club as argument and returns
    top 6,6-10,mid-table, relagation based on the club 
    finishing position for 2022. If club  was not in 
    the league in 2022 returns default value midtable
    
    """
    if club in standings_2022[:6]:
        return 'top 6'
   
    if club in standings_2022[6:11]:
        return '6-10'
    
    if club in standings_2022[11:16]:
        return 'mid-table'
    
    if club in standings_2022[16:] :
        return 'relegation'
    
    else:
        return 'mid-table'


In [83]:
def Region(country):
    
    """
    Function takes country as an argument and returns 
    the region of the country
    
    """
    
    south_america = {'uruguay', 'colombia', 'argentina', 'ecuador','chile', 'venezuela', 'paraguay','peru','bolivia'}
    europe = {'spain','italy','belarus','portugal','finland','ukraine','finland'}
    rest_of_world = {'south-korea','united-states','china','japan','cote-d-ivoire'}
    brazil={'brazil'}
    
    if country in south_america:
        return 'south america'
    
    elif country in europe:
        return 'europe'
    
    elif country in rest_of_world:
        return 'rest of the world'
    
    elif country in brazil:
        return 'brazil'
    
    else:
        return 'not in data'

In [84]:
def Age(age):
    """
    Function takes age as an argument and returns 
    the age range
    
    """
    if age <= 18:
        return 'U18'
    if age in range(18, 22):
        return '18-21'
    if age in range(22, 24):
        return '21-23'
    if age in range(24, 28):
        return '23-27'
    if age in range(28, 33):
        return '27-32'
    return '32+'

In [85]:
def Starter(starter):
    
     if starter =='Yes' :
        return '1'
     else:
        return '0'

In [86]:
def calculate_similarities(row,input_data):
    s = 0
    
    if row['club_code'] == input_data[0]:   # Check for same club
        s += 1
    
    if row['club_level'] == Club_type(input_data[0]):  # Check for simlar level club
        s += 0.5
    
    if row['player_group_code'] == input_data[1]:
        s += 4
    
    if row['player_country_id'] == input_data[2]:  # Check for same country
        s += 1
    
    if row['region'] == Region(input_data[2]):   # Check for same region
        s += 0.5
    
    if row['Age'] == Age(input_data[3]):
        s += 1
    
    if row['high_earner'] == Starter(input_data[4]):
        s += 8
    
    return s


In [87]:
def similar_player(data):
    
    X['similarities'] = X.apply(calculate_similarities, axis = 1,input_data = data)
    X_sorted = X.sort_values(by = 'similarities', ascending = False)
    top_5 = X_sorted.head(5)
    indexes = top_5.index

    
    return df_similar.loc[indexes]

In [88]:
similar_player(['AMG','D','brazil',34,'Yes'])

,season_id,player_name,club_code,player_age,player_country_id,player_group_code,salary_gross_eur,high_earner
658,2022,João Paulo,AMG,35,brazil,D,238147,0
645,2022,Danilo Avelar,AMG,33,brazil,D,400087,1
1474,2021,João Paulo,AMG,34,brazil,D,238147,1
674,2022,Patric,AMG,33,brazil,D,171466,0
1467,2021,Eduardo,AMG,34,brazil,D,28578,0


#### Additional function to get % spend on player

In [89]:
def Total_wage_bill(club):
    """
    function takes in club as argument and 
    returns the the total wage bill of the club
    
    """
    df_filtered = df_salary.loc[df_salary['club_code'] == club]
    
    #Number of seasons for which data is available for the club
    n = len(df_filtered.season_id.unique())
    
    #Assuming club has simlar squad sizes each season
    total = df_filtered['salary_gross_eur'].sum()/n
    
    return total

In [90]:
Total_wage_bill('AMG')

6015601.0

In [91]:
##Convert notebook to script run in terminal
# jupyter nbconvert --to script Brazil_Similar_players.ipynb

In [92]:
CODE_DICT


{'Athletico Paranaense': 'CAP',
 'Atletico Mineiro': 'CAM',
 'Botafogo': 'BOT',
 'Ceara Sporting Club': 'CEA',
 'Corinthians': 'CRS',
 'Coritiba': 'CRB',
 'Esporte Clube Bahia': 'ECB',
 'Flamengo': 'FLA',
 'Fluminense': 'FLU',
 'Fortaleza': 'FOR',
 'Goianiense': 'GOE',
 'Goias': 'GOI',
 'Gremio': 'GRM',
 'Internacional': 'INR',
 'Palmeiras': 'PLM',
 'Red Bull Bragantino': 'RED',
 'Santos': 'SAN',
 'Sao Paulo': 'SAP',
 'Sport Club do Recife': 'SCR',
 'Vasco da Gama': 'VAS',
 'America FC': 'AMG',
 'Avai': 'AVI',
 'Cuiaba': 'CUI',
 'Juventude': 'JUD',
 'Chapecoense': 'CPE',
 'Grêmio': 'GRM',
 'São Paulo': 'SAP',
 'Goiás': 'GOI',
 'Bahia': 'ECB',
 'Ceará': 'CEA',
 'Atlético Mineiro': 'CAM',
 'Atlético Goianiense': 'GOE',
 'Sport': 'SCR',
 'América Mineiro': 'AMG',
 'Cuiabá': 'CUI',
 'Avaí': 'AVI'}